# Motor Predictive Maintenance - Real-Time Data Analysis v2

## Data Format Support
This notebook automatically detects and handles both data formats:

**Format 1 (No timestamps):**
```
ax_g,ay_g,az_g,mic_raw,temp_C
```

**Format 2 (With timestamps):**
```
timestamp,ax_g,ay_g,az_g,mic_raw,temp_C
```

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.fft import fft, fftfreq
from datetime import datetime

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print('✓ Libraries loaded successfully')

## 1. Smart Data Loading (Auto-detects format)

In [ ]:
# Load data - automatically detect format
def load_motor_data(filename='prediction.csv'):
    """Smart loader that handles both formats"""
    
    # Try to read first line to detect format
    with open(filename, 'r') as f:
        first_line = f.readline().strip()
    
    # Count commas to determine number of columns
    num_cols = first_line.count(',') + 1
    
    if num_cols == 6:  # Has timestamps
        print("✓ Detected format: WITH timestamps")
        df = pd.read_csv(filename)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['reading_num'] = range(1, len(df) + 1)
        df['elapsed_sec'] = (df['timestamp'] - df['timestamp'].iloc[0]).dt.total_seconds()
        has_timestamps = True
        
    elif num_cols == 5:  # No timestamps
        print("✓ Detected format: WITHOUT timestamps")
        df = pd.read_csv(filename, header=None, names=['ax_g', 'ay_g', 'az_g', 'mic_raw', 'temp_C'])
        df['reading_num'] = range(1, len(df) + 1)
        has_timestamps = False
        
    else:
        raise ValueError(f"Unexpected format: {num_cols} columns found")
    
    return df, has_timestamps

# Load the data
df, has_timestamps = load_motor_data('prediction.csv')

print(f"\n📊 Data Summary")
print("=" * 60)
print(f"Total readings: {len(df)}")
print(f"Columns: {list(df.columns)}")
if has_timestamps:
    duration = df['elapsed_sec'].max()
    print(f"Duration: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"Sampling rate: {len(df)/duration:.2f} Hz")
    print(f"Start time: {df['timestamp'].iloc[0]}")
    print(f"End time: {df['timestamp'].iloc[-1]}")
print("=" * 60)

df.head(10)

## 2. Data Quality & Statistics

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())
print("\n" + "="*60 + "\n")

# Statistical summary
print("Statistical Summary:")
df[['ax_g', 'ay_g', 'az_g', 'mic_raw', 'temp_C']].describe()

In [ ]:
# Calculate vibration magnitude
df['vibration_magnitude'] = np.sqrt(df['ax_g']**2 + df['ay_g']**2 + df['az_g']**2)

# Identify anomalies
print("🚨 Anomaly Detection:\n")

# High vibration (>1.5g)
accel_threshold = 1.5
high_vib = df[df['vibration_magnitude'] > accel_threshold]

if len(high_vib) > 0:
    print(f"⚠️  High Vibration Events: {len(high_vib)} readings (>{accel_threshold}g)")
    if has_timestamps:
        display(high_vib[['timestamp', 'vibration_magnitude', 'ax_g', 'ay_g', 'az_g', 'temp_C']].head(10))
    else:
        display(high_vib[['reading_num', 'vibration_magnitude', 'ax_g', 'ay_g', 'az_g', 'temp_C']].head(10))
else:
    print("✅ No high vibration events detected")

# Microphone outliers
Q1 = df['mic_raw'].quantile(0.25)
Q3 = df['mic_raw'].quantile(0.75)
IQR = Q3 - Q1
mic_outliers = df[df['mic_raw'] > Q3 + 3 * IQR]  # Using 3*IQR for extreme outliers

if len(mic_outliers) > 0:
    print(f"\n⚠️  Microphone Anomalies: {len(mic_outliers)} readings")
    cols = ['timestamp', 'mic_raw', 'temp_C', 'vibration_magnitude'] if has_timestamps else ['reading_num', 'mic_raw', 'temp_C', 'vibration_magnitude']
    display(mic_outliers[cols].sort_values('mic_raw', ascending=False).head(10))

# Temperature extremes
temp_high = df[df['temp_C'] > 32]
if len(temp_high) > 0:
    print(f"\n🌡️  High Temperature Events: {len(temp_high)} readings (>32°C)")
    print(f"   Max: {df['temp_C'].max():.2f}°C")

## 3. Time Series Visualization

In [ ]:
# Determine x-axis (timestamp or reading number)
x_axis = 'elapsed_sec' if has_timestamps else 'reading_num'
x_label = 'Time (seconds)' if has_timestamps else 'Reading Number'

fig, axes = plt.subplots(5, 1, figsize=(16, 14))

# Accelerometer X
axes[0].plot(df[x_axis], df['ax_g'], marker='o', markersize=2, linewidth=1, alpha=0.7)
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0].axhline(y=1, color='r', linestyle='--', alpha=0.3, label='±1g')
axes[0].axhline(y=-1, color='r', linestyle='--', alpha=0.3)
axes[0].set_ylabel('Accel X (g)', fontsize=12)
axes[0].set_title('Accelerometer X-axis', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accelerometer Y
axes[1].plot(df[x_axis], df['ay_g'], marker='o', markersize=2, linewidth=1, alpha=0.7, color='orange')
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1].set_ylabel('Accel Y (g)', fontsize=12)
axes[1].set_title('Accelerometer Y-axis', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Accelerometer Z
axes[2].plot(df[x_axis], df['az_g'], marker='o', markersize=2, linewidth=1, alpha=0.7, color='green')
axes[2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2].set_ylabel('Accel Z (g)', fontsize=12)
axes[2].set_title('Accelerometer Z-axis', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# Microphone (log scale)
axes[3].plot(df[x_axis], df['mic_raw'], marker='o', markersize=2, linewidth=1, alpha=0.7, color='red')
axes[3].set_ylabel('Microphone\n(log scale)', fontsize=12)
axes[3].set_yscale('log')
axes[3].set_title('Microphone Signal', fontsize=14, fontweight='bold')
axes[3].grid(True, alpha=0.3, which='both')

# Temperature
axes[4].plot(df[x_axis], df['temp_C'], marker='o', markersize=3, linewidth=1.5, alpha=0.8, color='purple')
axes[4].axhline(y=30, color='orange', linestyle='--', alpha=0.5, label='30°C')
axes[4].set_ylabel('Temperature (°C)', fontsize=12)
axes[4].set_xlabel(x_label, fontsize=12)
axes[4].set_title('Temperature', fontsize=14, fontweight='bold')
axes[4].legend()
axes[4].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Vibration Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Vibration over time
axes[0].plot(df[x_axis], df['vibration_magnitude'], marker='o', markersize=2, linewidth=1, color='darkblue')
axes[0].axhline(y=df['vibration_magnitude'].mean(), color='green', linestyle='--', 
                alpha=0.7, label=f"Mean: {df['vibration_magnitude'].mean():.3f}g")
axes[0].axhline(y=df['vibration_magnitude'].mean() + 2*df['vibration_magnitude'].std(), 
                color='red', linestyle='--', alpha=0.5, label='Mean + 2σ')
axes[0].set_xlabel(x_label, fontsize=12)
axes[0].set_ylabel('Vibration Magnitude (g)', fontsize=12)
axes[0].set_title('Total Vibration Magnitude', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(df['vibration_magnitude'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[1].axvline(x=df['vibration_magnitude'].mean(), color='green', linestyle='--', linewidth=2, label='Mean')
axes[1].axvline(x=df['vibration_magnitude'].median(), color='orange', linestyle='--', linewidth=2, label='Median')
axes[1].set_xlabel('Vibration Magnitude (g)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Vibration Statistics:")
print(f"  Mean: {df['vibration_magnitude'].mean():.4f}g")
print(f"  Std:  {df['vibration_magnitude'].std():.4f}g")
print(f"  Max:  {df['vibration_magnitude'].max():.4f}g")

## 5. Time-Based Analysis (if timestamps available)

In [ ]:
if has_timestamps:
    print("⏱️  TIME-BASED ANALYSIS\n")
    
    # Calculate sampling rate
    time_diffs = df['timestamp'].diff().dt.total_seconds()
    avg_interval = time_diffs.mean()
    actual_sampling_rate = 1 / avg_interval if avg_interval > 0 else 0
    
    print(f"Actual Sampling Rate: {actual_sampling_rate:.2f} Hz")
    print(f"Average interval: {avg_interval:.3f} seconds")
    print(f"Min interval: {time_diffs.min():.3f} seconds")
    print(f"Max interval: {time_diffs.max():.3f} seconds")
    
    # Plot sampling intervals
    fig, ax = plt.subplots(figsize=(16, 4))
    ax.plot(df['elapsed_sec'][1:], time_diffs[1:], marker='.', markersize=3, alpha=0.7)
    ax.axhline(y=avg_interval, color='red', linestyle='--', label=f'Average: {avg_interval:.3f}s')
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Sampling Interval (s)', fontsize=12)
    ax.set_title('Sampling Interval Over Time', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calculate rates of change
    df['temp_rate'] = df['temp_C'].diff() / time_diffs
    df['vibration_rate'] = df['vibration_magnitude'].diff() / time_diffs
    
    print(f"\n📈 Rates of Change:")
    print(f"  Temperature: {df['temp_rate'].mean():.4f} °C/s")
    print(f"  Max temp increase: {df['temp_rate'].max():.4f} °C/s")
    print(f"  Max temp decrease: {df['temp_rate'].min():.4f} °C/s")
    
else:
    print("ℹ️  No timestamps available. Add timestamps for time-based analysis.")
    print("   Use the provided data_collection_with_timestamps.py script!")

## 6. Correlation Analysis

In [ ]:
corr_cols = ['ax_g', 'ay_g', 'az_g', 'mic_raw', 'temp_C', 'vibration_magnitude']
corr_matrix = df[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Sensor Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 7. Summary Report

In [ ]:
print("="*70)
print(" MOTOR HEALTH SUMMARY REPORT")
print("="*70)

print(f"\n📊 Dataset:")
print(f"   Total Readings: {len(df)}")
if has_timestamps:
    print(f"   Duration: {df['elapsed_sec'].max()/60:.1f} minutes")
    print(f"   Sampling Rate: {actual_sampling_rate:.2f} Hz")

print(f"\n🔧 Vibration:")
print(f"   Mean: {df['vibration_magnitude'].mean():.4f}g")
print(f"   Std: {df['vibration_magnitude'].std():.4f}g")
print(f"   Max: {df['vibration_magnitude'].max():.4f}g")
high_count = len(df[df['vibration_magnitude'] > 1.5])
print(f"   High Events (>1.5g): {high_count} ({high_count/len(df)*100:.1f}%)")

print(f"\n🌡️  Temperature:")
print(f"   Current: {df['temp_C'].iloc[-1]:.2f}°C")
print(f"   Mean: {df['temp_C'].mean():.2f}°C")
print(f"   Range: {df['temp_C'].min():.2f}°C - {df['temp_C'].max():.2f}°C")

print(f"\n🎤 Microphone:")
print(f"   Median: {df['mic_raw'].median():.0f}")
print(f"   Max: {df['mic_raw'].max():.0f}")
print(f"   Outliers: {len(mic_outliers)}")

print(f"\n💡 Status:")
if high_count > 5:
    print("   ⚠️  CAUTION: Multiple high vibration events detected")
elif high_count > 0:
    print("   🟡 NORMAL: Some vibration spikes observed")
else:
    print("   ✅ GOOD: Vibration within normal range")

if df['temp_C'].max() > 35:
    print("   🔴 HIGH TEMPERATURE WARNING")
elif df['temp_C'].max() > 32:
    print("   🟡 Elevated temperature observed")
else:
    print("   ✅ Temperature nominal")

print("\n" + "="*70)

## 8. Export Data

In [ ]:
# Save processed data
df.to_csv('motor_data_processed.csv', index=False)
print('✓ Processed data saved: motor_data_processed.csv')

# Save summary
summary = df[['ax_g', 'ay_g', 'az_g', 'mic_raw', 'temp_C', 'vibration_magnitude']].describe()
summary.to_csv('motor_data_summary.csv')
print('✓ Summary saved: motor_data_summary.csv')